In [17]:
import numpy as np
import cv2 as cv
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import copy

================================== Part One - Shi-Thomas ==================================

In [27]:
video_address = r"../Input/book_low.mp4"
video = cv.VideoCapture(video_address)
ret, previous_frame = video.read()
previous_image = cv.cvtColor(previous_frame, cv.COLOR_BGR2GRAY)
previous_image = cv.GaussianBlur(previous_image, ksize=(15, 15), sigmaX=2, sigmaY=2)
colored_image = copy.deepcopy(previous_frame)

max_corners = 50
quality_level = 0.3
min_distance = 10
block_size = 10
mask = None

points = cv.goodFeaturesToTrack(previous_image, maxCorners=max_corners, qualityLevel=quality_level, minDistance=min_distance, mask=None, blockSize=block_size, useHarrisDetector=False)
for i in range(len(points)):
    cv.circle(colored_image, (int(points[i][0][0]), int(points[i][0][1])), 3, (0, 127, 250), -1)

save_address ="../Outputs/Q1/best_result.png"
# save_address ="../Outputs/Q1/" +"mx_corners_"+str(max_corners)+"_quality_level_"+str(quality_level)+"_min_distance_"+str(min_distance)+"_block_size_"+str(block_size)+".png"

cv.imshow("Output", colored_image)
cv.imwrite(save_address, colored_image)
cv.waitKey(0)
cv.destroyAllWindows()

================================== Part Two - Kanade ==================================

In [29]:
fourcc = cv.VideoWriter_fourcc(*'MP4V')
out = cv.VideoWriter('output_Kanade.mp4', fourcc, 30.0, (previous_frame.shape[1], previous_frame.shape[0]))
count = 1
endpoints_new = []
points_previous = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    
    new_image = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    points, state, err = cv.calcOpticalFlowPyrLK(previous_image, new_image, points, None,winSize  = (20, 20),maxLevel = 25,criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
    previous_image = new_image.copy()
    endpoints_next = []

    if len(points) > 0:
        for i,status in enumerate(state):
            if status == 1:
                endpoints_next.append(points[i])
            else:
                endpoints_next.append(None)
    
    if len(endpoints_next) > 0 and len(endpoints_new) > 0:
        for old,new in zip(endpoints_new,endpoints_next):
            if old is not None and new is not None:
                frame = cv.arrowedLine(frame,  old[0].astype(int),  new[0].astype(int), (255,255,0), 1, tipLength = 0.5)
    
    if len(endpoints_new) > 0 and len(points_previous) > 0:
        for old, new in zip(points_previous, endpoints_new):
            if old is not None and new is not None:
                frame = cv.arrowedLine(frame, old[0].astype(int), new[0].astype(int), (0, 255, 255), 1, tipLength = 0.5)

    points_previous = endpoints_new.copy()
    endpoints_new = endpoints_next.copy()

    cv.imshow('result', frame)
    out.write(frame)

    if(count % 15 == 0):
        save_address ="../Outputs/Q2/" + str(count) + ".png"
        cv.imwrite(save_address, frame)
    count += 1

    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

out.release()
cv.destroyAllWindows()

================================== Part Three - Farneback ==================================

In [25]:
video = cv.VideoCapture(video_address)
_, frame = video.read()
previous_image = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
output_HSV = np.zeros_like(frame)
output_HSV[..., 1] = 255
count = 1
fourcc = cv.VideoWriter_fourcc(*'MP4V')
out_Farneback = cv.VideoWriter('output_Farneback.mp4', fourcc, 30.0, (previous_image.shape[1], previous_image.shape[0]))

while(1):
    ret, frame = video.read()
    if not ret:
        break
    
    new_image = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(previous_image, new_image, flow, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    output_HSV[..., 0] = ang*180/np.pi/2
    output_HSV[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    output_BGR = cv.cvtColor(output_HSV, cv.COLOR_HSV2BGR)

    cv.imshow('output', output_BGR)

    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

    out_Farneback.write(output_BGR)
    if(count % 15 == 0):
        save_address ="../Outputs/Q3/" + str(count) + ".png"
        cv.imwrite(save_address, output_BGR)
    count += 1
    previous_image = new_image

cv.destroyAllWindows()
out_Farneback.release()